In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset = dataset.astype(int)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [11]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [13]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [21]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_pa

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [23]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [25]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.8805970149253731


In [27]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[77  8]
 [ 8 41]]


In [29]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.91      0.91      0.91        85
           1       0.84      0.84      0.84        49

    accuracy                           0.88       134
   macro avg       0.87      0.87      0.87       134
weighted avg       0.88      0.88      0.88       134



In [31]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.8713085234093638

In [33]:
table=pd.DataFrame.from_dict(re)

In [35]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001116,0.001774,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000201,0.000402,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.002840,0.001706,0.005784,0.003392,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.782557,0.889022,0.814409,0.851527,0.865054,0.840514,0.037728,4
3,0.001962,0.001745,0.015321,0.003793,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.826263,0.831253,0.831098,0.783837,0.885265,0.831543,0.032218,5
4,0.008351,0.007148,0.007241,0.004919,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.808927,0.868752,0.814409,0.849057,0.865054,0.841240,0.025097,2
5,0.003121,0.002337,0.005245,0.001035,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.808927,0.795789,0.813179,0.766556,0.885265,0.813943,0.039212,8
6,0.001207,0.001473,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000985,0.001257,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.002619,0.002351,0.005693,0.000898,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.831253,0.777291,0.906166,0.862442,0.840683,0.042607,3
9,0.002483,0.002284,0.004368,0.002390,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.812243,0.889022,0.740173,0.832483,0.883278,0.831440,0.054226,6


In [37]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 38
BMI: 29
Children: 1
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 1


In [39]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

ValueError: X has 5 features, but DecisionTreeClassifier is expecting 3 features as input.